Examples of all below models being used can be found here: https://github.com/timeseriesAI/tsai/tree/main/nbs

Stats:
- 15 models return promising eval stats
- 5 models return bad eval stats (0.5 <= AUC <= 0.56)
- 6 models hasn't been trained yet due to errors (ROCKETS & Tab models)

In [1]:
import tsai.all
import datetime
import pandas as pd
import torch
import numpy as np

In [2]:
%run ../shared_functions.py

In [3]:
%run ../my_shared_functions.py

<Figure size 640x480 with 0 Axes>

In [4]:
SEED = 42

if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

seed_everything(SEED)

Selected device is cuda


In [5]:
DIR_INPUT = '../../fraud-detection-handbook/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 547 ms
Wall time: 4.85 s
919767 transactions loaded, containing 8195 fraudulent transactions


In [6]:
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7


delta_valid = delta_test

start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))

(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training_with_valid,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)

# standardizing
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

SEQ_LEN = 5

def prepare_sequenced_X_y(df, seq_len, input_features, output_feature):
    x = torch.FloatTensor(df[input_features].values) # shape => [66928, 15] for train
    y = torch.FloatTensor(df[output_feature].values)
    features = torch.vstack([x, torch.zeros(x[0,:].shape)]) # shape => [66929, 15] for train
    df_ids_dates = pd.DataFrame({'CUSTOMER_ID':df['CUSTOMER_ID'].values,
            'TX_DATETIME':df['TX_DATETIME'].values})
    df_ids_dates["tmp_index"]  = np.arange(len(df_ids_dates))
    df_groupby_customer_id = df_ids_dates.groupby("CUSTOMER_ID")
    sequence_indices = pd.DataFrame(
        {
            "tx_{}".format(n): df_groupby_customer_id["tmp_index"].shift(SEQ_LEN - n - 1)
            for n in range(SEQ_LEN)
        }
    )

    sequences_ids = sequence_indices.fillna(len(features) - 1).values.astype(int) # shape => [66928, 5] for train

    x_sequenced = [features[sequences_ids[index], :].transpose(0, 1) for index in range(x.shape[0])]
    return torch.stack(x_sequenced), y # x shape => [66928, 15, 5] for train

x_train, y_train = prepare_sequenced_X_y(train_df, SEQ_LEN, input_features, output_feature)
x_valid, y_valid = prepare_sequenced_X_y(valid_df, SEQ_LEN, input_features, output_feature)

In [7]:
X, y, splits = tsai.all.combine_split_data([x_train.numpy(), x_valid.numpy()], [y_train.numpy(), y_valid.numpy()])

In [8]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True) # https://timeseriesai.github.io/tsai/data.core.html#tsdatasets
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=64, num_workers=0, drop_last=True, device=DEVICE)

#### LSTM

In [8]:
learn = tsai.all.ts_learner(dls, tsai.all.LSTM, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.033948,0.022591,00:09


In [9]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.704,0.328,0.54,0.187


#### GRU

In [10]:
learn = tsai.all.ts_learner(dls, tsai.all.GRU, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.029032,0.020899,00:08


In [11]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.73,0.401,0.6,0.196


In [18]:
dls.bs, dls.c, dls.vars, dls.len

(64, 1, 15, 5)

#### MLP

In [13]:
model = tsai.all.MLP(dls.vars, dls.c, dls.len, layers=[500, 500, 500])
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.039753,0.023390,00:09


In [14]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.722,0.356,0.569,0.19


#### FCN

In [14]:
learn = tsai.all.ts_learner(dls, tsai.all.FCN, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.028863,0.020305,00:14


In [15]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.712,0.388,0.578,0.181


#### ResNet

In [16]:
learn = tsai.all.ts_learner(dls, tsai.all.ResNet, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.023652,0.019332,00:28


In [17]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.762,0.433,0.639,0.219


#### LSTM-FCN

In [18]:
learn = tsai.all.ts_learner(dls, tsai.all.LSTM_FCN, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.025815,0.020302,00:14


In [19]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.742,0.413,0.616,0.201


#### GRU-FCN

In [20]:
learn = tsai.all.ts_learner(dls, tsai.all.GRU_FCN, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.023219,0.020373,00:15


In [21]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.764,0.414,0.63,0.224


#### mWDN (multilevel Wavelet Decomposition Network)


##### Level=1

In [66]:
model = tsai.all.mWDN(dls.vars, dls.c, dls.len, levels=1)

In [67]:
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.021926,0.019964,00:41


In [68]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.766,0.433,0.642,0.226


##### Level=2

In [69]:
model = tsai.all.mWDN(dls.vars, dls.c, dls.len, levels=2)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.023441,0.020700,00:42


In [70]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.767,0.408,0.627,0.231


##### Level=3

In [72]:
model = tsai.all.mWDN(dls.vars, dls.c, dls.len, levels=3)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time


RuntimeError: Given input size: (15x1x1). Calculated output size: (15x1x0). Output size is too small

##### Level=4

In [73]:
model = tsai.all.mWDN(dls.vars, dls.c, dls.len, levels=4)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time


RuntimeError: Given input size: (15x1x1). Calculated output size: (15x1x0). Output size is too small

#### TCN (Temporal Convolution Network) (BAD RESULT AUC=0.5)

returns AUC=0.5 without seed_everything with 42 (or any other number)

returns AUC=0.5 with seed_everything with 42 (and train time~50min + pred time~30min)

In [12]:
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=64, num_workers=0, drop_last=True, device=DEVICE)
model = tsai.all.TCN(dls.vars, dls.c)

learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=DEVICE, lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.044169,0.032110,50:56


In [13]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.5,0.006,0.0,0.011


#### InceptionTime

In [10]:
learn = tsai.all.ts_learner(dls, tsai.all.InceptionTime, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.024124,0.019191,00:41


In [11]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.754,0.414,0.624,0.214


#### XceptionTime (BAD RESULT AUC=0.501)

In [95]:
model = tsai.all.XceptionTime(dls.vars, dls.c)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.693178,0.693152,00:35


In [30]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.501,0.009,0.005,0.013


#### ResCNN

In [31]:
learn = tsai.all.ts_learner(dls, tsai.all.ResCNN, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.023283,0.019197,00:19


In [32]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.767,0.445,0.649,0.226


#### OmniScaleCNN

In [34]:
learn = tsai.all.ts_learner(dls, tsai.all.OmniScaleCNN, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.026753,0.020335,02:59


In [35]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.743,0.399,0.609,0.2


#### TST (Time Series Transformer)

In [36]:
learn = tsai.all.ts_learner(dls, tsai.all.TST, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.027224,0.018671,00:31


In [37]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.752,0.452,0.644,0.206


#### XCM (Explainable CNN for Multivariate TSC)

In [101]:
model = tsai.all.XCM(dls.vars, dls.c, dls.len)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.028300,0.020293,00:15


In [102]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.731,0.4,0.601,0.199


#### gMLP (gated MLP)

In [42]:
learn = tsai.all.ts_learner(dls, tsai.all.gMLP, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.022136,0.018877,00:37


In [43]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.758,0.458,0.651,0.204


#### XResNet1d

In [46]:
learn = tsai.all.ts_learner(dls, tsai.all.xresnet1d50_deeper, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.031698,0.022521,02:47


In [47]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.727,0.368,0.58,0.197


#### TransformerModel (BAD RESULT AUC=0.521)

In [8]:
model = tsai.all.TransformerModel(dls.vars, dls.c)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.030536,0.027414,00:18


In [9]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.521,0.025,0.078,0.023


#### TSPerceiver (BAD RESULT AUC=0.5)

In [7]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
# batch_size=16 is the maximum, with batch_size>=32 there's an error about VRAM being too closely to be exceeded
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=16, num_workers=0, drop_last=True, device=torch.device('cuda'))
model = tsai.all.TSPerceiver(dls.vars, dls.c, dls.len)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,0.036220,0.027919,10:15


In [8]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.5,0.006,0.0,0.011


#### TSiT (based on ViT) (BAD RESULT AUC=0.557)

In [10]:
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=64, num_workers=0, drop_last=True, device=torch.device('cuda'))
model = tsai.all.TSiTPlus(dls.vars, dls.c, dls.len)
learn = tsai.all.ts_learner(dls, model, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

use_token set to False as c_out == 1


epoch,train_loss,valid_loss,time
0,0.035824,0.024733,00:55


In [11]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.557,0.088,0.198,0.036


#### Rocket example

Pytorch (GPU) implementation: (c_in, seq_len)

https://timeseriesai.github.io/tsai/models.rocket_pytorch.html

https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/02_ROCKET_a_new_SOTA_classifier.ipynb#scrollTo=iUafIHSiYvrB

In [8]:
Xx, yx, splitsx = tsai.all.get_UCR_data('HandMovementDirection', split_data=False)
tfmsx  = [None, [tsai.all.Categorize()]]
batch_tfmsx = [tsai.all.TSStandardize(by_sample=True)]
dlsx = tsai.all.get_ts_dls(Xx, yx, splits=splitsx, tfms=tfmsx, drop_last=False, shuffle_train=False, batch_tfms=batch_tfmsx, bs=10_000)

In [9]:
Xx.shape, yx.shape

((234, 10, 400), (234,))

In [24]:
np.count_nonzero(Xx==None)

0

In [25]:
np.count_nonzero(yx==None)

0

In [26]:
np.count_nonzero(Xx==0)

0

In [18]:
yx

memmap(['right', 'right', 'right', 'right', 'right', 'right', 'right',
        'right', 'right', 'right', 'right', 'right', 'right', 'right',
        'right', 'right', 'right', 'right', 'right', 'right', 'right',
        'right', 'right', 'right', 'right', 'right', 'right', 'right',
        'right', 'right', 'right', 'right', 'right', 'right', 'right',
        'right', 'right', 'right', 'right', 'right', 'forward', 'forward',
        'forward', 'forward', 'forward', 'forward', 'forward', 'forward',
        'forward', 'forward', 'forward', 'forward', 'forward', 'forward',
        'forward', 'forward', 'forward', 'forward', 'forward', 'forward',
        'forward', 'forward', 'forward', 'forward', 'forward', 'forward',
        'forward', 'forward', 'forward', 'forward', 'forward', 'forward',
        'forward', 'forward', 'forward', 'forward', 'forward', 'forward',
        'forward', 'forward', 'left', 'left', 'left', 'left', 'left',
        'left', 'left', 'left', 'left', 'left', 'left', 

In [17]:
model = tsai.all.build_ts_model(tsai.all.ROCKET, dls=dlsx)
X_trainx, y_trainx = tsai.all.create_rocket_features(dlsx.train, model)
X_validx, y_validx = tsai.all.create_rocket_features(dlsx.valid, model)
X_trainx.shape, X_validx.shape

((160, 20000), (74, 20000))

#### Rocket (REQUIRES FIX)

(CPU-Numpy Rocket version works fine without a problem)

In [13]:
X.shape, y.shape

((125433, 15, 5), (125433,))

In [27]:
np.count_nonzero(X==None)

0

In [23]:
np.count_nonzero(X==0)

1326315

In [28]:
np.count_nonzero(y==None)

0

In [87]:
tfmsx  = [None, [tsai.all.Categorize()]]
batch_tfmsx = [tsai.all.TSStandardize(by_sample=True)]
dls_rocket = tsai.all.get_ts_dls(X, y, splits=splits, tfms=tfmsx, drop_last=False, shuffle_train=False, batch_tfms=batch_tfmsx)
model = tsai.all.build_ts_model(tsai.all.ROCKET, dls=dls_rocket, n_kernels=1_000)
# model = tsai.all.build_ts_model(tsai.all.ROCKET, dls=dls_rocket, **{'n_kernels':10000, 'kss':[7, 9, 11]})
X_train_rocket, y_train_rocket = tsai.all.create_rocket_features(dls_rocket.train, model)
X_valid_rocket, y_valid_rocket = tsai.all.create_rocket_features(dls_rocket.valid, model)
X_train_rocket.shape, X_valid_rocket.shape

ValueError: 'a' cannot be empty unless no samples are taken

In [94]:
next(iter(dls_rocket.loaders[0]))

(TSTensor(samples:64, vars:15, len:5, device=cuda:0, dtype=torch.float32),
 TensorCategory([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                 0], device='cuda:0'))

In [93]:
next(iter(dls_rocket.loaders[0]))[0].detach().cpu().numpy()

array([[[ 0.16747907,  0.16747907,  0.16747907,  0.16747907,
         -0.1118022 ],
        [ 0.16747907,  0.16747907,  0.16747907,  0.16747907,
         -1.1642246 ],
        [ 0.16747907,  0.16747907,  0.16747907,  0.16747907,
          4.772523  ],
        ...,
        [ 0.16747907,  0.16747907,  0.16747907,  0.16747907,
         -0.09250171],
        [ 0.16747907,  0.16747907,  0.16747907,  0.16747907,
         -1.8848672 ],
        [ 0.16747907,  0.16747907,  0.16747907,  0.16747907,
         -0.13576384]],

       [[-0.17787611, -0.17787611, -0.17787611, -0.17787611,
          0.19716749],
        [-0.17787611, -0.17787611, -0.17787611, -0.17787611,
         -1.7444433 ],
        [-0.17787611, -0.17787611, -0.17787611, -0.17787611,
          5.239328  ],
        ...,
        [-0.17787611, -0.17787611, -0.17787611, -0.17787611,
         -0.48370793],
        [-0.17787611, -0.17787611, -0.17787611, -0.17787611,
          2.7944007 ],
        [-0.17787611, -0.17787611, -0.17787611, 

In [ ]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

#### MiniRocket (REQUIRES FIX)

In [97]:
dls_minirocket = tsai.all.get_ts_dls(X, y, splits=splits, bs=256)
learn = tsai.all.ts_learner(dls_minirocket, tsai.all.MiniRocket, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,nan,nan,00:02


In [98]:
valid_dl = learn.dls.valid
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.5,0.006,0.0,0.011


#### MiniRocketVoting (no PyTorch implementation) (REQUIRES FIX)

In [100]:
model = tsai.all.MiniRocketVotingClassifier()
model.fit(x_train.numpy(), y_train.numpy())

ValueError: n_timepoints must be >= 9, but found 5; zero pad shorter series so that n_timepoints == 9

#### TabModel (REQUIRES FIX)

In [33]:
learn = tsai.all.ts_learner(dls, tsai.all.TabModel, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

TypeError: __init__() missing 1 required positional argument: 'c_out'

#### TabTransformer (REQUIRES FIX)

In [38]:
learn = tsai.all.ts_learner(dls, tsai.all.TabTransformer, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

TypeError: __init__() missing 1 required positional argument: 'c_out'

#### GatedTabTransformer (TabTransformer + gMLP) (REQUIRES FIX)

In [44]:
learn = tsai.all.ts_learner(dls, tsai.all.GatedTabTransformer, loss_func=tsai.all.BCEWithLogitsLossFlat(), device=torch.device('cuda'), lr=0.001, metrics=[])
learn.fit_one_cycle(1)

TypeError: __init__() missing 1 required positional argument: 'c_out'